In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
from urllib.request import urlopen
import requests
import bs4
import pandas as pd

### 동적 실행중에 동적으로 실행된 값 추출
 - 당연히도 정적 페이지에서는 추출 불가능
 - selenium을 사용하여 동적으로 변경된 페이지를 따로 추출한다

In [2]:
url = 'https://n.news.naver.com/mnews/article/079/0003812365?sid=100'
htmls = urlopen(url)
bs_obj = bs4.BeautifulSoup(htmls, 'html.parser')

In [3]:
# 정적 : 제목 추출
title = bs_obj.find('div', {'class':'media_end_head_title'}).find('span').text
title

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

In [4]:
# 동적 : 좋아요 추출 (안됨)
like_num = bs_obj.find('span', {'class':'u_likeit_text _count num'})
if like_num is None:
    print('값이 없습니다')

값이 없습니다


In [6]:
# comment_count 수 추출 (안됨)
comment_count =  bs_obj.find('a', {'id':'comment_count'}).text
comment_count

'댓글'

#### selenium 패키지를 사용한 자동 크롤링
 - 근데 서드파티라 설치를 따로 해야함
 - 터미널 창에서 !pip install selenium

In [7]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/f9/2f/9c6eef6487faca5006ae1ba43cf6ab627c7e3d2a7ec5a3b8728e2105472d/selenium-4.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/a3/dd/b61fa61b186d3267ef3903048fbee29132963ae762fb70b08d4a3cd6f7aa/trio-0.22.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/f2/25/868488d8b05104e3d1bf162565bbadd5fe6d62bf903710e6402c3aaceb1e/trio_websocket-0.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for exceptiongroup from https://files.pythonhosted.org/packages/ad/83/b71e58666f156a39fb29417e4c8ca4bc7400c0dd4ed9e8842ab54dc8c344/exceptiongroup-1.1.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 43.8 MB/s eta 0:00:0000:0100:01

In [8]:
!pip show selenium

Name: selenium
Version: 4.12.0
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: /Users/jangseungheon/anaconda3/lib/python3.11/site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 


In [10]:
!pip install webdriver_manager

  Obtaining dependency information for webdriver_manager from https://files.pythonhosted.org/packages/19/5a/a16653bfce685c9832217d377f52065351eeac9862e44e2996cd81f3bb4d/webdriver_manager-4.0.0-py2.py3-none-any.whl.metadata


In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
# (1) webdriver 객체 생성 / webdriver가 제어하는 크롬창 하나 생성
# 주의! 드라이버를 통해서 열고 닫지 않는다면 오류발생!
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [13]:
# 페이지 접속
driver.get(url)

In [16]:
# 동적 : 좋아요 추출
# 속성명 사이에는 스페이스가 존재하면 안되고 전부 .으로 대체한다
like_num = driver.find_element(By.CSS_SELECTOR, '.u_likeit_text._count.num')
like_num.text

like_num = driver.find_element(By.CLASS_NAME, 'u_likeit_text._count.num')
like_num.text

'2'

'2'

In [18]:
# comment_count 수 추출 (안됨)
comment_count = driver.find_element(By.ID, 'comment_count')
comment_count.text

'36'

In [27]:
# selenium 사용 가능
paper = driver.find_element(By.CSS_SELECTOR, '.media_end_head_top_logo img')
paper.get_attribute('title') # title 속성값 추출

title = driver.find_element(By.CSS_SELECTOR, '.media_end_head_title').text
title

datetime = driver.find_element(By.CSS_SELECTOR, '.media_end_head_info_datestamp_time').text
datetime

'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [20]:
# 정적 크롤링 추출
paper = bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title']
paper

title = bs_obj.find('div', {'class':'media_end_head_title'}).find('span').text
title

timestamp = bs_obj.find('div', {'class':'media_end_head_info_datestamp_bunch'}).find('span').text
timestamp

'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [31]:
# find_elements() : 여러 객체를 리스트로 반환
dates = driver.find_elements(By.CSS_SELECTOR, '.media_end_head_info_datestamp div')
for d in dates:
    print(d.text)

1

입력2023.09.13. 오후 3:07


In [32]:
# 페이지 닫음
driver.close()

In [ ]:
# selenium 메소드
# driver.find_element(By.CLASS_NAME, "클래스명")
# driver.find_element(By.CSS_SELECTOR, "선택자 양식")
# driver.find_element(By.ID, "ID명")
# driver.find_element(By.LINK_TEXT, "Selenium Official Page")
# driver.find_element(By.NAME, "newsletter")
# driver.find_element(By.PARTIAL_LINK_TEXT, "Official Page")
# driver.find_element(By.TAG_NAME, "a")
# driver.find_element(By.XPATH, "//input[@value='f']")
# # driver가 접근한 페이지의 source 추출
# html = driver.page_source